Exercício 10.2 

In [1]:
from si.io.csv_file import read_csv
from si.model_selection.split import train_test_split
from si.models.knn_classifier import KNNClassifier
from si.models.logistic_regression import LogisticRegression
from si.models.decision_tree_classifier import DecisionTreeClassifier
from si.ensemble.stacking_classifier import StackingClassifier
from si.metrics.accuracy import accuracy


In [5]:
# 1) carregar dataset
dataset = read_csv("datasets/breast-bin.csv")

TypeError: read_csv() missing 3 required positional arguments: 'sep', 'features', and 'label'

In [ ]:
# 2) split treino/teste
train_ds, test_ds = train_test_split(dataset, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

In [ ]:
# 3) modelos base
knn = KNNClassifier(n_neighbors=5)
logreg = LogisticRegression()
dt = DecisionTreeClassifier(max_depth=5)

NameError: name 'KNNClassifier' is not defined

In [ ]:
# 4) modelo final
final_knn = KNNClassifier(n_neighbors=3)

NameError: name 'KNNClassifier' is not defined

In [ ]:
# 5) stacking
stacking = StackingClassifier(
    models=[knn, logreg, dt],
    final_model=final_knn
)

NameError: name 'StackingClassifier' is not defined

In [ ]:
# 6) treino
stacking.fit(train_ds)


NameError: name 'stacking' is not defined

In [ ]:
# 7) avaliação
y_pred = stacking.predict(test_ds)
score = accuracy(test_ds.y, y_pred)

print("Accuracy do StackingClassifier:", score)

NameError: name 'stacking' is not defined